In [ ]:
import os
import json

import pickle
import pandas as pd
import numpy as np

In [ ]:
FOLDER_PATH = os.getcwd()
input_file_name = "results/aisourcing_20220102_20220103_output_checkpoint-3133.pkl"
input_file_name = "results/aisourcing_20220102_20220103_output_checkpoint-6266.pkl"

input_file_path = os.path.join(FOLDER_PATH, input_file_name)

In [ ]:
# df = pd.read_csv(input_file_path)
with open(input_file_path, 'rb') as file:
    df = pickle.load(file)

In [ ]:
df.index.size

In [ ]:
df.head()

In [ ]:
def reverse_sigmoid(arr):
    arr = arr.numpy()
    return np.log(arr / (1 - arr))

In [ ]:
df['predicted_logits'] = df['predicted_label'].apply(lambda x: reverse_sigmoid(x))

In [ ]:
def softmax(arr):
    # print(arr)
    return np.exp(arr) / np.sum(np.exp(arr))

In [ ]:
softmax(df['predicted_logits'][0][0])

In [ ]:
df['predicted_softmax'] = df['predicted_logits'].apply(lambda x: softmax(x[0]))

In [ ]:
df['predicted_softmax']

In [ ]:
df['predicted_ones'] = df['predicted_softmax'].apply(lambda x: x[1])
df['predicted_zeros'] = df['predicted_softmax'].apply(lambda x: x[0])

In [ ]:
df['predicted_softmax']

In [ ]:
from sklearn.metrics import roc_auc_score

In [ ]:
roc_auc_score(df['label'], df['predicted_ones'])

In [ ]:
roc_auc_score(df['label'], df['predicted_zeros'])

In [ ]:
groups = df.groupby("task_id")

In [ ]:
gauc = 0.0
for _, entry in groups:
    tmp_auc = roc_auc_score(entry['label'], entry['predicted_ones'])
    weight = float(len(entry))
    
    gauc += weight * tmp_auc

gauc /= float(len(df))

In [ ]:
gauc